In [32]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 26 09:58:21 2022

@author: tabubakirov
"""

import ps, ps_utils
import numpy as np
import pandas as pd
import datetime as dt
ps_utils.switch_to_prod_pvlss()

# Static data
t = dt.date(2023, 5, 31) # On which date override is performed
# Constant override
md_override_hash = '1b182fda53a2e8eea2a7a0ec749ecd891f86fd84' 
md_override = ps.get(md_override_hash)
mds = ps_utils.market.get_mds(t)
mds = ps_utils.market.mds_apply_overrides(mds, md_override) # Applying constant override
md_source = '31-05-2023_SourceTableWithMarketData.xlsx'
#md_source = 'MarketData_FX_IR.xlsx'

# Boolean values in order to understand, what market data to replace
is_override_all = False
is_override_credit_curves = False
is_override_ir_curves = True
is_override_ir_vols = False
is_override_fx_vols = False
is_override_com_curves = False

# Replacing market data
md_override = ps.new_cm()

# Credit curves
if is_override_credit_curves or is_override_all:
    md_override['CreditCurves'] = {}
    
    curves_df = pd.read_excel(md_source, 'CreditCurves')
    unique_curves = curves_df.Curve.unique()
    for curve_name in unique_curves:
        curve_ps = mds.CreditCurves[curve_name]
        curve_df = curves_df.loc[curves_df['Curve'] == curve_name]
        curve_ps.Quotes.Values = curve_df['Values'].tolist()
        curve_ps.RatesSchedule.Rates = curve_df['Rates'].tolist()
        md_override.CreditCurves[curve_name] = curve_ps

# IR Curves
if is_override_ir_curves or is_override_all:
    md_override['RatesCurvesBundles'] = {}
    
    curves_df = pd.read_excel(md_source, 'IRCurves')
    unique_curves = curves_df.Curve.unique()
    for curve_name in unique_curves:
        curve_ps = mds.RatesCurvesBundles[curve_name]
        curve_df = curves_df.loc[curves_df['Curve'] == curve_name]
        
        # Redefining curve
        n = len(curve_ps.RatesSchedule.Dates)
        for i in range(0, n):
            date = curve_ps.RatesSchedule.Dates[i]
            curve_sub_df = curve_df.loc[curve_df['Date'] == date.strftime('%Y-%m-%d')]
            if not curve_sub_df.empty:
                disc_rate = curve_sub_df.iloc[0]['Rate']
                curve_ps.RatesSchedule.Rates[i] = disc_rate
        
        # Saving curve
        md_override.RatesCurvesBundles[curve_name] = curve_ps


# IR Vols
if is_override_all or is_override_ir_vols:
    if 'VolatilitySurfaces' not in md_override.keys():
        md_override['VolatilitySurfaces'] = {}
    
    surfaces = ['RUB_KEY_RATE', 'RUB_MOSPRIME_3M'] # List here the surfaces to be overriden
    # Names of surfaces should correspond both: their names in Excel source and PS
    # market data object
    for surface_name in surfaces:
        # Reading target surface from Excel
        surface_df = pd.read_excel(md_source, 'IRVol_' + surface_name)
        
        #Getting current surface from PS
        surface_ps = mds.VolatilitySurfaces[surface_name]
        tenors = surface_ps.Tenors
        strikes = surface_ps.Strikes
        new_surface = []
        
        # Preparing target surface to be assigned in PS
        for tenor in tenors:
            new_surface.append(list(surface_df[tenor] / 100))
        
        # Saving target surface in PS
        surface_ps.VolatilityQuotes = new_surface
        md_override.VolatilitySurfaces[surface_name] = surface_ps
    

# FX Vols
if is_override_fx_vols or is_override_all:
    if 'VolatilitySurfaces' not in md_override.keys():
        md_override['VolatilitySurfaces'] = {}
    
    surfaces = ['USDRUB', 'EURRUB', 'XAUUSD', 'CNHRUB'] # List here the surfaces to be overriden
    # Names of surfaces should correspond both: their names in Excel source and PS
    # market data object
    for surface_name in surfaces:
        # Reading target surface from Excel
        surface_df = pd.read_excel(md_source, 'FXVol_' + surface_name)
        
        #Getting current surface from PS
        surface_ps = mds.VolatilitySurfaces[surface_name]
        tenors = surface_ps.Tenors
        deltas = surface_ps.Deltas
        new_surface = []
        
        # Preparing target surface to be assigned in PS
        for tenor in tenors:
            row_df = surface_df[surface_df['Tenor'] == tenor]
            vols = list(map(lambda delta: row_df[delta].tolist()[0], deltas))
            new_surface.append(vols)
        
        # Saving target surface in PS
        surface_ps.VolatilityQuotes = new_surface
        md_override.VolatilitySurfaces[surface_name] = surface_ps

# Commodity curves
if is_override_com_curves or is_override_all:
    md_override['Forwards'] = {}
    
    curves_df = pd.read_excel(md_source, 'ComCurves')
    unique_curves = curves_df.Curve.unique()
    for curve_name in unique_curves:
        curve_ps = mds.Forwards[curve_name]
        curve_df = curves_df.loc[curves_df['Curve'] == curve_name]
        
        # Redefining curve
        n = len(curve_ps.ForwardSchedule.Dates)
        for i in range(0, n):
            date = curve_ps.ForwardSchedule.Dates[i]
            curve_sub_df = curve_df.loc[curve_df['Date'] == date.strftime('%Y-%m-%d')]
            if not curve_sub_df.empty:
                price = curve_sub_df.iloc[0]['Quote']
                curve_ps.ForwardSchedule.ForwardQuotes[i] = price
        
        # Saving curve
        md_override.Forwards[curve_name] = curve_ps

mds = ps_utils.market.mds_apply_overrides(mds, md_override)
print(mds._id)


[INFO] 2023-06-05 21:40:26,225 - Nothing to do: current environment is already prod_pvlss


8d4dbddb5863ed49a46220530cca541a9f123e8e


In [47]:
r=ps.get('8c19cdbfaf6567d0015b8e649299c66a029faa63')
r.CreditCurves['RUB_']._id

'2eac8bb2dbd9dc5fd1a2fee89524c8d8c80718f3'

In [49]:
r=ps.get('6bd86ed711f77f3036b30d53b367f2a7867a3dd2')
r.CreditCurves['RUB_MINFIN_OFZ']._id

'2eac8bb2dbd9dc5fd1a2fee89524c8d8c80718f3'

# CS01: спред россии, сбера 51, и северстали 63bp оверрайд 72886080d143636ae0e69976d471e90b050f0335

In [43]:
#resp_published_31-05-2023
bcva_fo=0
published_resp_hash=ps.get('67dabf2628afbba9d08c1c543cb420b0409cab19')
for el in published_resp_hash.Result:
    try:
        bcva_fo+=el.Result['BCVA']
    except:
        continue
print("BCVA_FO=",bcva_fo)


#reqv_hash=ps.get(published_resp_hash._nonce)
reqv_hash=ps.get('e93a9ab90b2ec4e4e00e06ff017fe6a6eb88b39c')
for el in reqv_hash:
    el.Model.MarketDataSet='72886080d143636ae0e69976d471e90b050f0335'
    el.Queries=['BCVA']
try:
    ps_utils.compute(ps.put(reqv_hash),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
except:
    print("Some failures")
    
bcva_ipv=0
ipv_resp_hash=ps.get('89a9da436230b9a95bd938b882751842a1b29be8')
for el in ipv_resp_hash.Result:
    try:
        bcva_ipv+=el.Result['BCVA']
    except:
        continue
print("BCVA_IPV=",bcva_ipv)
print("Diff=",bcva_ipv-bcva_fo)

BCVA_FO= -17636195.80973125


[INFO] 2023-06-06 11:01:36,349 - Compute:b4fe9813e2e15d4547cc1788141e0455452158ea->89a9da436230b9a95bd938b882751842a1b29be8


Some failures
BCVA_IPV= -15760153.441206252
Diff= 1876042.3685249966


In [30]:
# Тимур еще RPZSO добавил
1876042+115000

1991042

In [44]:
published=-17636195
published_with_rpzso=-17753951
with_cs01_curves_and_rpzso=-15760153
with_cs01_curves_and_rpzso-published_with_rpzso

1993798

# FXVega оверрайд USDRUB EURRUB CNHRUB XAUUSD 37874a0d59198d955ef2f45623d88a57108cd667

In [3]:
#resp_published_31-05-2023
bcva_fo=0
published_resp_hash=ps.get('67dabf2628afbba9d08c1c543cb420b0409cab19')
for el in published_resp_hash.Result:
    try:
        bcva_fo+=el.Result['BCVA']
    except:
        continue
print("BCVA_FO=",bcva_fo)


#reqv_hash=ps.get(published_resp_hash._nonce)
reqv_hash=ps.get('e93a9ab90b2ec4e4e00e06ff017fe6a6eb88b39c')
for el in reqv_hash:
    el.Model.MarketDataSet='37874a0d59198d955ef2f45623d88a57108cd667'
    el.Queries=['BCVA']
try:
    ps_utils.compute(ps.put(reqv_hash),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
except:
    print("Calculation finished")
    
bcva_ipv=0
ipv_resp_hash=ps.get('eec6787bddcd0b16232ed9882b32ab7845170ca8')
for el in ipv_resp_hash.Result:
    try:
        bcva_ipv+=el.Result['BCVA']
    except:
        continue
print("BCVA_IPV=",bcva_ipv)
print("Diff=",bcva_ipv-bcva_fo)

BCVA_FO= -17636195.80973125


[INFO] 2023-06-05 20:08:43,769 - Compute:9025651311c5f22764f66d36de8b3d0aa927611a->eec6787bddcd0b16232ed9882b32ab7845170ca8


Calculation finished
BCVA_IPV= -17636365.412996475
Diff= -169.6032652258873


In [4]:
r=ps.get('37874a0d59198d955ef2f45623d88a57108cd667')
print(r.VolatilitySurfaces['USDRUB']._id)
print(r.VolatilitySurfaces['EURRUB']._id)
print(r.VolatilitySurfaces['CNHRUB']._id)
print(r.VolatilitySurfaces['XAUUSD']._id)

53a1a657c5afa506145e753a18677eaf2f331344
47ea97be914c6ee325e16c2048320fb24d77e344
c450b99e07104e49132fd1e76bbab90ca4e51db1
3d956473ac4e0f7d98d195cfcfc3796c7c086230


In [8]:
r=ps.get(ps.get('854040bf42f451e089a1db2399db6c022ea24f70')._nonce)[0].Model.MarketDataSet
print(r.VolatilitySurfaces['USDRUB']._id)
print(r.VolatilitySurfaces['EURRUB']._id)
print(r.VolatilitySurfaces['CNHRUB']._id)
print(r.VolatilitySurfaces['XAUUSD']._id)

53a1a657c5afa506145e753a18677eaf2f331344
47ea97be914c6ee325e16c2048320fb24d77e344
c450b99e07104e49132fd1e76bbab90ca4e51db1
3d956473ac4e0f7d98d195cfcfc3796c7c086230


In [10]:
-17752033+17636195

-115838

# IR Vol оверрайд RUB_KEY_RATE и RUB_MOSPRIME 3faed7dd801bfda8878a9971ac2ce3c8749eba70

# Vega level

In [22]:
#resp_published_31-05-2023
bcva_fo=0
published_resp_hash=ps.get('67dabf2628afbba9d08c1c543cb420b0409cab19')
for el in published_resp_hash.Result:
    try:
        bcva_fo+=el.Result['BCVA']
    except:
        continue
print("BCVA_FO=",bcva_fo)


#reqv_hash=ps.get(published_resp_hash._nonce)
reqv_hash=ps.get('e93a9ab90b2ec4e4e00e06ff017fe6a6eb88b39c')

for el in reqv_hash:
    el.Model.MarketDataSet='3faed7dd801bfda8878a9971ac2ce3c8749eba70'
    el.Queries=['BCVA']
try:
    ps_utils.compute(ps.put(reqv_hash),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
except:
    print("Calculation finished")
    
bcva_ipv=0
ipv_resp_hash=ps.get('1f6581ca48c0b93619a735bce5eaabe68471fd5c')
for el in ipv_resp_hash.Result:
    try:
        bcva_ipv+=el.Result['BCVA']
    except:
        continue
print("BCVA_IPV=",bcva_ipv)
print("Diff=",bcva_ipv-bcva_fo)

BCVA_FO= -17636195.80973125


[INFO] 2023-06-05 20:55:33,370 - Compute:f17cdef217d367eb6e0a087b5457ca3899d3cc32->1f6581ca48c0b93619a735bce5eaabe68471fd5c


Calculation finished
BCVA_IPV= -17806638.852474913
Diff= -170443.04274366423


In [14]:
#ipv calculated in orchestrator
-17804019+17636195

-167824

# Vega wo Smile but with ipv vols

In [24]:
#resp_published_31-05-2023
bcva_fo=0
published_resp_hash=ps.get('67dabf2628afbba9d08c1c543cb420b0409cab19')
for el in published_resp_hash.Result:
    try:
        bcva_fo+=el.Result['BCVA']
    except:
        continue
print("BCVA_FO=",bcva_fo)


#reqv_hash=ps.get(published_resp_hash._nonce)
reqv_hash=ps.get('e93a9ab90b2ec4e4e00e06ff017fe6a6eb88b39c')

for el in reqv_hash:
    el.Model.MarketDataSet='3faed7dd801bfda8878a9971ac2ce3c8749eba70'
    el.Queries=['BCVA']
try:
    ps_utils.compute(ps.put(reqv_hash),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
except:
    print("Calculation finished")
    
bcva_ipv=0
ipv_resp_hash=ps.get('b7a14869a3315c2115f01689af64f014716d012c')
for el in ipv_resp_hash.Result:
    try:
        bcva_ipv+=el.Result['BCVA']
    except:
        continue
print("BCVA_IPV=",bcva_ipv)
print("Diff=",bcva_ipv-bcva_fo)

BCVA_FO= -17636195.80973125


[INFO] 2023-06-05 21:00:31,368 - Compute:f17cdef217d367eb6e0a087b5457ca3899d3cc32->1f6581ca48c0b93619a735bce5eaabe68471fd5c


Calculation finished
BCVA_IPV= -17690971.14740001
Diff= -54775.33766876161


# Apply Smile

In [25]:
mas_cpties=[]
bcva_fo=[]
bcva_smile=[]
r=ps.get('1f6581ca48c0b93619a735bce5eaabe68471fd5c')
for el in r.Result:
    if(ps.get(el._nonce)['_Counterparty'] in ['FRTGS','MKSMO','OSKOI','RAZYU']):
        #print(ps.get(el._nonce)['_Counterparty'],el.Result['BCVA'])
        z=ps.get(el._nonce)
        z.Model.TypeName='HybridCheyette1FactorLocalVolatilityModel'
        z.Queries=['ProductValue','BCVA']
        otv=ps_utils.compute(ps.put(z),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
        #print(ps.get(el._nonce)['_Counterparty'],otv['BCVA'])
        bcva_fo.append(el.Result['BCVA'])
        bcva_smile.append(otv['BCVA'])
        mas_cpties.append(ps.get(el._nonce)['_Counterparty'])
print("BCVA_FO=",np.sum(bcva_fo))
print("Bcva_Smile=",np.sum(bcva_smile))
print("Diff=",np.sum(bcva_smile)-np.sum(bcva_fo))
df=pd.DataFrame({'Cpty':mas_cpties,'NoSmile':bcva_fo,'WithSmile':bcva_smile})
df

[INFO] 2023-06-05 21:00:47,641 - Compute:193c96eb3f1941d6d25911a76021e0033c023b59->08d5f3851248d7670eaef6103c60e7f9e109bcfd
[INFO] 2023-06-05 21:00:48,373 - Compute:33a5a7ef0127d8119777075e5fe5da5000ffa88d->3571c4f23f96bce6541b7feb8405b2f6c08a0cbb
[INFO] 2023-06-05 21:00:49,231 - Compute:8d6a3a388c6d413057790f1da15ab3a720e50646->ee6ebac5f5c595c4d04c5952931514f4a64a8be1
[INFO] 2023-06-05 21:00:51,774 - Compute:c6565dce7268c6a124556f15c2acc0028cdfb7bb->76bc159fb7085a8d06eae4a53cfa20dad92e96ac


BCVA_FO= -5470253.556941899
Bcva_Smile= -5924623.896712796
Diff= -454370.3397708973


,Cpty,NoSmile,WithSmile
0,MKSMO,"-677,670.22","-1,202,757.63"
1,FRTGS,"-4,401,856.29","-4,448,907.35"
2,RAZYU,"-333,517.19","-225,928.66"
3,OSKOI,"-57,209.86","-47,030.25"


In [33]:
#MKSMO PV for 27-04-2023
(-1634793508+2688499146-364685978)/81.3775

8466955.36235446

In [26]:
# ipv adj IRVega murex
bulo=20446023
stalo=20496278
stalo-bulo

50255

# DV01 оверрайд XAU: USD, EUR: xccy, usd_xccy, CNH_FX, inr_fx  оверрайд 8d4dbddb5863ed49a46220530cca541a9f123e8e

In [36]:
#resp_published_31_05-2023
bcva_fo=0
published_resp_hash=ps.get('67dabf2628afbba9d08c1c543cb420b0409cab19')
for el in published_resp_hash.Result:
    try:
        bcva_fo+=el.Result['BCVA']
    except:
        continue
print("BCVA_FO=",bcva_fo)


reqv_hash=ps.get(published_resp_hash._nonce)
#reqv_hash=ps.get('e93a9ab90b2ec4e4e00e06ff017fe6a6eb88b39c')
for el in reqv_hash:
    el.Model.MarketDataSet='8d4dbddb5863ed49a46220530cca541a9f123e8e'
    el.Queries=['BCVA']
try:
    ps_utils.compute(ps.put(reqv_hash),algo_id='833aa9286a2951a0eac3fb38913d6990435e5c4c')
except:
    print("Calculation finished")
    
bcva_ipv=0
ipv_resp_hash=ps.get('54c86a7b29002956f7a9cbe07200502ae0212800')
for el in ipv_resp_hash.Result:
    try:
        bcva_ipv+=el.Result['BCVA']
    except:
        continue
print("BCVA_IPV=",bcva_ipv)
print("Diff=",bcva_ipv-bcva_fo)

BCVA_FO= -17636195.80973125


[INFO] 2023-06-05 21:51:11,437 - Compute:9f29c23cbc5ad05b8983d7b586bce100d30f1481->54c86a7b29002956f7a9cbe07200502ae0212800


Calculation finished
BCVA_IPV= -17679204.892773572
Diff= -43009.08304232359


# Ful reval овеерайд 8c19cdbfaf6567d0015b8e649299c66a029faa63

In [51]:
#resp_published_28-04-2023
bcva_fo=0
published_resp_hash=ps.get('b623cdb556f20253c11a84aa7e2b17759be2ce48')
for el in published_resp_hash.Result:
    try:
        bcva_fo+=el.Result['BCVA']
    except:
        continue
print("BCVA_FO=",bcva_fo)


reqv_hash=ps.get(published_resp_hash._nonce)
for el in reqv_hash:
    el.Model.MarketDataSet='8c19cdbfaf6567d0015b8e649299c66a029faa63'
    el.Queries=['BCVA']
try:
    ps_utils.compute(ps.put(reqv_hash))
except:
    print("Calculation finished")
    
bcva_ipv=0
ipv_resp_hash=ps.get('a1467755fd32a93f506d4764e2eddd7711302905')
for el in ipv_resp_hash.Result:
    try:
        bcva_ipv+=el.Result['BCVA']
    except:
        continue
print("BCVA_IPV=",bcva_ipv)
print("Diff=",bcva_ipv-bcva_fo)

BCVA_FO= -18124786.01944381


[INFO] 2023-05-05 11:26:12,494 - Compute:0bce0c1edadbcd40d20e5432f42052f3aa1735f4->a1467755fd32a93f506d4764e2eddd7711302905


Calculation finished
BCVA_IPV= -16940903.34327687
Diff= 1183882.6761669405
